In [ ]:
!git clone https://github.com/CodingYodha/QnA-bot-using-GRU.git

Cloning into 'QnA-bot-using-GRU'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 8.84 KiB | 2.95 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [ ]:
!pip install convokit --no-deps


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for convokit: filename=convokit-3.1.0-py3-none-any.whl size=229062 sha256=ba6d8a648fa1b45472907488812b64e14546396e1b2b0b7345b4486b4cbdbe25
  Stored in directory: /root/.cache/pip/wheels/3d/6c/a6/ff02f7442da7b87607faba9e518247c4db5d68f7ed39c2f610
Successfully built convokit


In [ ]:
import pandas as pd
from convokit import Corpus, download

# Download and load the corpus
corpus = Corpus(filename=download("movie-corpus"))
corpus.print_summary_stats()

# Extract utterance data
data = []
for utt in corpus.iter_utterances():
    data.append({
        "conversation_id": utt.conversation_id,
        "utterance_id": utt.id,
        "speaker": utt.speaker.id,
        "text": utt.text
    })




An error occurred: No module named 'cleantext'
No configuration file found at /root/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
model_directory: ~/.convokit/saved-models
default_backend: mem


/usr/local/lib/python3.10/dist-packages/convokit/__init__.py:26: UserWarning: If you are using ConvoKit with Google Colab, incorrect versions of some packages (ex. scipy) may be imported while runtime start. To fix the issue, restart the session and run all codes again. Thank you!
  warnings.warn(


Number of Speakers: 9035
Number of Utterances: 304713
Number of Conversations: 83097


In [ ]:
# Group utterances by conversation_id
from collections import defaultdict

conversations = defaultdict(list)
for entry in data:
    conversations[entry['conversation_id']].append((entry['utterance_id'], entry['speaker'], entry['text']))

# Extract question-answer pairs
questions = []
answers = []

for conv_id, utterances in conversations.items():
    for i in range(len(utterances) - 1):
        current_utterance = utterances[i][2]  # Current utterance text
        next_utterance = utterances[i + 1][2]  # Next utterance text
        questions.append(current_utterance)
        answers.append(next_utterance)

# Example output
print("Question:", questions[100])
print("Answer:", answers[100])

Question: What do you think?
Answer: Oh, I thought you might have a date  I don't know why I'm bothering to ask, but are you going to Bogey Lowenstein's party Saturday night?


In [ ]:
import re
def clean_text(text):
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"[^a-zA-Z?.!]+", " ", text)
    text = text.strip()
    return text.lower()

questions = [clean_text(q) for q in questions]
answers = [clean_text(a) for a in answers]

# Add <start> and <end> tokens to answers
answers = ['<start> ' + a + ' <end>' for a in answers]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Tokenizer
tokenizer = Tokenizer(num_words=10000,filters="", oov_token="<OOV>")
tokenizer.fit_on_texts(questions + answers)

In [ ]:
# Vocabulary size
VOCAB_SIZE = 10000#len(tokenizer.word_index) + 1

In [ ]:
# Convert text to sequences
question_sequences = tokenizer.texts_to_sequences(questions)
answer_sequences = tokenizer.texts_to_sequences(answers)

In [ ]:
# Pad sequences
MAX_LENGTH = 19  # Adjust based on your data
question_padded = pad_sequences(question_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')
answer_padded = pad_sequences(answer_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import GRU, Dense, Embedding, Input, Attention
from tensorflow.keras.models import Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Attention, Concatenate
from tensorflow.keras.models import Model

# Assume VOCAB_SIZE is defined based on your tokenizer
VOCAB_SIZE = 10000 #len(tokenizer.word_index) + 1
MAX_LENGTH = 19  # as used for encoder inputs
DECODER_LENGTH = MAX_LENGTH - 1  # decoder expects sequences without the last token

# Encoder
encoder_inputs = Input(shape=(MAX_LENGTH,), name='encoder_inputs')
enc_emb = Embedding(VOCAB_SIZE, 256, name='encoder_embedding')(encoder_inputs)
encoder_gru = GRU(256, return_sequences=True, return_state=True,
                  dropout=0.0, recurrent_dropout=0.0, unroll=True,
                  name='encoder_gru')
encoder_outputs, encoder_state = encoder_gru(enc_emb)

# Decoder
decoder_inputs = Input(shape=(DECODER_LENGTH,), name='decoder_inputs')
dec_emb = Embedding(VOCAB_SIZE, 256, name='decoder_embedding')(decoder_inputs)
decoder_gru = GRU(256, return_sequences=True, return_state=True,
                  dropout=0.0, recurrent_dropout=0.0, unroll=True,
                  name='decoder_gru')
decoder_outputs, _ = decoder_gru(dec_emb, initial_state=encoder_state)

# Attention Layer
attention_layer = Attention(name='attention_layer')
attention_output = attention_layer([decoder_outputs, encoder_outputs])

# Concatenate attention output with decoder outputs
decoder_concat = Concatenate(axis=-1, name='decoder_concat')([decoder_outputs, attention_output])

# Dense output layer
decoder_dense = Dense(VOCAB_SIZE, activation='softmax', name='decoder_dense')
output = decoder_dense(decoder_concat)

# Build and compile the model
model = Model([encoder_inputs, decoder_inputs], output)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, 19)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, 18)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_embedding         │ (None, 19, 256)        │      2,560,000 │ encoder_inputs[0][0]   │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_embedding         │ (None, 18, 256)        │      2,560,000 │ decoder_inputs[0][0]   │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_gru (GRU)         │ [(None, 19, 256),      │        394,752 │ encoder_embedding[0][… │
│                           │ (None, 256)]           │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_gru (GRU)         │ [(None, 18, 256),      │        394,752 │ decoder_embedding[0][… │
│                           │ (None, 256)]           │                │ encoder_gru[0][1]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_layer           │ (None, 18, 256)        │              0 │ decoder_gru[0][0],     │
│ (Attention)               │                        │                │ encoder_gru[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_concat            │ (None, 18, 512)        │              0 │ decoder_gru[0][0],     │
│ (Concatenate)             │                        │                │ attention_layer[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_dense (Dense)     │ (None, 18, 10000)      │      5,130,000 │ decoder_concat[0][0]   │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 11,039,504 (42.11 MB)

 Trainable params: 11,039,504 (42.11 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
len(tokenizer.word_index)

99488

In [ ]:
encoder_input_data = question_padded
decoder_input_data = answer_padded[:, :-1]  # Remove <end> token
decoder_output_data = answer_padded[:, 1:]

In [ ]:
invalid_indices = np.where((decoder_output_data < 0) | (decoder_output_data >= VOCAB_SIZE))

if len(invalid_indices[0]) > 0:
    print("Invalid label tokens found!")
    print("Locations:", invalid_indices)
    print("Values at those locations:", decoder_output_data[invalid_indices])
else:
    print("All label tokens are in the valid range [0, {}].".format(VOCAB_SIZE - 1))

All label tokens are in the valid range [0, 9999].


In [ ]:
# Prepare inputs and outputs
from tensorflow.keras.callbacks import ReduceLROnPlateau
encoder_input_data = question_padded
decoder_input_data = answer_padded[:, :-1]  # Remove <end> token
decoder_output_data = answer_padded[:, 1:]   # Remove <start> token

# Train the model
BATCH_SIZE = 64
EPOCHS = 12
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=1, verbose=1)

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_output_data,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.2,
    callbacks=[reduce_lr]
)

Epoch 1/12
2771/2771 ━━━━━━━━━━━━━━━━━━━━ 1077s 389ms/step - accuracy: 0.5626 - loss: 2.8431 - val_accuracy: 0.5829 - val_loss: 2.6700 - learning_rate: 1.0000e-04
Epoch 2/12
2771/2771 ━━━━━━━━━━━━━━━━━━━━ 1055s 381ms/step - accuracy: 0.5921 - loss: 2.5715 - val_accuracy: 0.5899 - val_loss: 2.5706 - learning_rate: 1.0000e-04
Epoch 3/12
2771/2771 ━━━━━━━━━━━━━━━━━━━━ 1065s 384ms/step - accuracy: 0.5976 - loss: 2.4827 - val_accuracy: 0.5930 - val_loss: 2.5170 - learning_rate: 1.0000e-04
Epoch 4/12
2771/2771 ━━━━━━━━━━━━━━━━━━━━ 1073s 387ms/step - accuracy: 0.6008 - loss: 2.4220 - val_accuracy: 0.5949 - val_loss: 2.4803 - learning_rate: 1.0000e-04
Epoch 5/12
2771/2771 ━━━━━━━━━━━━━━━━━━━━ 1073s 387ms/step - accuracy: 0.6012 - loss: 2.3883 - val_accuracy: 0.5969 - val_loss: 2.4523 - learning_rate: 1.0000e-04
Epoch 6/12
2771/2771 ━━━━━━━━━━━━━━━━━━━━ 1079s 389ms/step - accuracy: 0.6039 - loss: 2.3552 - val_accuracy: 0.5982 - val_loss: 2.4319 - learning_rate: 1.0000e-04
Epoch 7/12
2771/2771 ━

In [ ]:
# Save model in HDF5 format (includes architecture, weights, optimizer state, etc.)
model.save("seq2seq_model_gru.h5")




In [ ]:
# Alternatively, save in TensorFlow's SavedModel format:
tf.saved_model.save(model, "saved_seq2seq_model_gru")